Hierbij wordt gebruik gemaakt van het package youtube-dl

In [1]:
import numpy as np
import pandas as pd
import re
import csv
import datetime
import os
import wave
import subprocess
import glob
import youtube_dl

In [2]:
## Lees alle csv-files in
bal_labels = pd.read_csv('csv_files/balanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

unbal_labels = pd.read_csv('csv_files/unbalanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

eval_labels = pd.read_csv('csv_files/eval_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [3]:
## Hierin worden de wav-files gegooid
file_path = "wav_files"

In [4]:
## Maak folder aan als deze nog niet bestaat
if not os.path.isdir(file_path):
    os.makedirs(file_path)
    
## Maak ook folders aan voor bal/unbal/eval (om later makkelijk labels weer op te halen)
if not os.path.isdir(file_path + "/bal"):
    os.makedirs(file_path + "/bal")
if not os.path.isdir(file_path + "/unbal"):
    os.makedirs(file_path + "/unbal")
if not os.path.isdir(file_path + "/eval"):
    os.makedirs(file_path + "/eval")

In [5]:
## Geef aan of de tf-records voor jungle of urban moeten worden gedownload (ik neem aan dat we alleen jungle doen)
target = "jungle" #"urban"
mid_to_label = pd.read_csv("csv_files/class_labels_indices_" + target + ".csv", sep=";")

In [6]:
mid_to_label

,index,mid,display_name
0,0,/t/dd00134,"Car, boat, motorcyle, truck"
1,1,/m/01j4z9,Chainsaw
2,2,/m/07pczhz,Chop
3,3,/m/032s66,"Gunshot, gunfire"
4,4,/m/09ct_,Helicopter
5,5,/t/dd00003,Male singing
6,6,/m/07rpkh9,Cattle
7,0,/m/02rlv9,"Car, boat, motorcyle, truck"
8,0,/m/04_sv,"Car, boat, motorcyle, truck"
9,7,/m/06bz3,Radio


In [7]:
def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [20]:
## Download wav-files van youtube

def downloadWav(vid_to_mid, folder):

    for i in range(vid_to_mid.shape[0]):
        mids = vid_to_mid.loc[i, "positive_labels"]
        labels = getLabels(mids)
    
        # Check of labels relevant zijn
        if(len(labels)>0):
            # DOWNLOAD PART
            url = vid_to_mid.loc[i, "YTID"]
            file_name = 'VID_' + url
            file_path = 'wav_files/' + folder
            
            ydl_opts = {
                'extractaudio': True,
                'format': 'bestaudio',
                'noplaylist': True,
                'outtmpl': file_path + file_name + '.%(ext)s',
#                 'quiet': True
            }
            
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download(['https://www.youtube.com/watch?v=' + url])
                                 
            # retrieve start of audio segment
            start_sec = vid_to_mid.loc[i, "start_seconds"]
            
            
            # PROCESSING PART (could be made seperately)
            files = glob.glob(file_path + file_name + '.*')
            print(file_path + file_name + '.*')

            for f in files:
                print(f)
                file = f.replace('\\','/')
                path = file.rsplit('.',1)[0]

                subprocess.call('ffmpeg -ss '+ str(int(start_sec)) + ' -t 10 -i "'+ file +'" -vn -acodec pcm_s16le -ar 16000 "'+ path +'.wav"', shell=True);
            
                # TODO remove source file
#                 os.remove(file)

In [21]:
## Hier wordt de functie aangeroepen en gaat het downloaden beginnen
downloadWav(bal_labels, "bal/")
#downloadWav(unbal_labels, "unbal/")
#downloadWav(eval_labels, "eval/")

[youtube] --aO5cdqSAg: Downloading webpage
[youtube] --aO5cdqSAg: Downloading video info webpage
[download] wav_files\bal\VID_--aO5cdqSAg.webm has already been downloaded
[download] 100% of 2.05MiB
wav_files/bal/VID_--aO5cdqSAg.*
wav_files/bal\VID_--aO5cdqSAg.wav
wav_files/bal\VID_--aO5cdqSAg.webm
[youtube] --aaILOrkII: Downloading webpage
[youtube] --aaILOrkII: Downloading video info webpage
[download] wav_files\bal\VID_--aaILOrkII.webm has already been downloaded
[download] 100% of 3.25MiB
wav_files/bal/VID_--aaILOrkII.*
wav_files/bal\VID_--aaILOrkII.wav
wav_files/bal\VID_--aaILOrkII.webm
[youtube] -24dqQM_rDk: Downloading webpage
[youtube] -24dqQM_rDk: Downloading video info webpage
[download] wav_files\bal\VID_-24dqQM_rDk.webm has already been downloaded
[download] 100% of 2.96MiB
wav_files/bal/VID_-24dqQM_rDk.*
wav_files/bal\VID_-24dqQM_rDk.wav
wav_files/bal\VID_-24dqQM_rDk.webm
[youtube] -3IGxVTJvgI: Downloading webpage
[youtube] -3IGxVTJvgI: Downloading video info webpage
[downl

KeyboardInterrupt: 

In [ ]:
# TEST check file (length)
#f = wave.open(file_path + '.wav', 'r')
#frames = f.getnframes()
#rate = f.getframerate()
#duration = frames / float(rate)
#f.close()

#if(duration<10):
#    print("Warning - duration of video '" + url + "' is only " + str(duration) + " second(s)!")